In [8]:
import os, json, pickle, csv, pandas
import numpy as np
from classDeclarations import file_data
from random import sample
MAX_SAMPLE_SIZE = 100

In [9]:
Path = "../../CDIACPub8"

imgs = []
tabular = []
freetext = []
json_xml = []
netcdf = []
unknown = []

df = pandas.read_csv('../CDIACFileData/labels/cdiac_naivetruth_processed.csv')

pathSeriesObj = df['path'].values
labelSeriesObj = df['file_label'].values

for i in range(len(labelSeriesObj)):
    if labelSeriesObj[i] == 'image':
        imgs.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'tabular':
        tabular.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'freetext':
        freetext.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'json/xml':
        json_xml.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'netcdf':
        netcdf.append(pathSeriesObj[i])
    else:
        unknown.append(pathSeriesObj[i])


In [10]:
img_sample_names = sample(imgs, MAX_SAMPLE_SIZE)
tabular_sample_names = sample(tabular, MAX_SAMPLE_SIZE)
freetext_sample_names = sample(freetext, MAX_SAMPLE_SIZE)
json_xml_sample_names = sample(json_xml, MAX_SAMPLE_SIZE)
netcdf_sample_names = sample(netcdf, MAX_SAMPLE_SIZE)
unknown_sample_names = sample(unknown, MAX_SAMPLE_SIZE)

In [11]:
print("loading files now...")
one_gram = json.load(open('../CDIACFileData/one_grams_distr_256.json', 'r'))
two_gram = json.load(open('../CDIACFileData/two_grams_distr_256.json', 'r'))

byte_distr = pandas.DataFrame.from_dict(one_gram)
two_grams_dicts = pandas.DataFrame.from_dict(two_gram)
#best_extractors = json.load(open("../CorrelatingExtractors/best_extractors.json"))
print("loading files done!")

loading files now...
loading files done!


In [12]:
byte_distr_new_index = byte_distr.index.tolist()
byte_distr_new_index.append("file_type")
byte_distr_new_index.append("file_size")
byte_distr = byte_distr.reindex(byte_distr_new_index)

two_grams_dicts_new_index = two_grams_dicts.index.tolist()
two_grams_dicts_new_index.append("file_type")
two_grams_dicts_new_index.append("file_size")
two_grams_dicts = two_grams_dicts.reindex(two_grams_dicts_new_index)

#byte_distr.sort_index(key=lambda x: x.str.lower())
#two_grams_dicts.sort_index(key=lambda x: x.str.lower())

In [13]:
img_data = []
tabular_data = []
freetext_data = []
json_xml_data = []
netcdf_data = []
unknown_data = []

dataset = [img_data, tabular_data, freetext_data, json_xml_data, netcdf_data, unknown_data]
for idx, data_list in enumerate(dataset):
    curr_sample = []
    if idx == 0:
        curr_sample = img_sample_names
        best_extractor = 'image'
    elif idx == 1:
        curr_sample = tabular_sample_names
        best_extractor = 'tabular'
    elif idx == 2:
        curr_sample = freetext_data
        best_extractor = 'freetext'
    elif idx == 3:
        curr_sample = json_xml_data
        best_extractor = 'json/xml'
    elif idx == 4:
        curr_sample = netcdf_data
        best_extractor = 'netcdf'
    else:
        curr_sample = unknown_data
        best_extractor = 'unknown'
    
    print("Done with index", idx)

    for file_name in curr_sample:
        #best_extractor_row_index = df.path[df.path==file_name].index.tolist()

        byte_distr.at['file_type', file_name] = idx 
        two_grams_dicts.at['file_type', file_name] = idx
      
        byte_distr.at['file_size', file_name] = os.path.getsize(file_name)
        two_grams_dicts.at['file_size', file_name] = os.path.getsize(file_name)

        curr_file_data = file_data(file_name, byte_distr[file_name], two_grams_dicts[file_name], best_extractor)
        data_list.append(curr_file_data)

Done with index 0
Done with index 1
Done with index 2
Done with index 3
Done with index 4
Done with index 5


In [14]:
print('Dumping!')

with open('gathered_data_appended_256KB_v2.pkl', 'wb+') as handle:
    pickle.dump(dataset, file=handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Dumped.')



Dumping!
Dumped.
